# 📥 Notebook 01 – Téléchargement des Spectres DR5

Ce notebook télécharge un lot de spectres .fits.gz du catalogue LAMOST DR5 à l’aide du script `dr5_downloader.py`.  
Il est conçu pour être exécuté **seulement lorsque de nouveaux spectres sont nécessaires**, afin d’alimenter le répertoire `data/raw/` en données brutes.

## ⚙️ Paramètres principaux

- `limit` : Nombre de plans de spectres à télécharger (chaque plan contient plusieurs spectres).
- `max_spectres` : Nombre maximum de spectres à télécharger.

## 🔗 Dépendances

- Le script Python `../src/tools/dr5_downloader.py` doit être présent et opérationnel.
- La configuration du projet doit pointer vers le bon répertoire `data/raw/`.

---

**Exécution** : Lance la cellule ci-dessous pour télécharger de nouveaux spectres depuis le catalogue public DR5.


In [ ]:
import sys
import subprocess
import ipywidgets as widgets
from IPython.display import display
import os
import datetime

# Paramètres interactifs
limit_widget = widgets.IntText(value=5, description='Plans :')
max_spectres_widget = widgets.IntText(value=500, description='Spectres :')
display(limit_widget, max_spectres_widget)

# Bouton d'exécution
run_button = widgets.Button(description="Lancer le téléchargement", button_style='success')
output_area = widgets.Output()
display(run_button, output_area)

def on_run_clicked(b):
    with output_area:
        output_area.clear_output()  # Clear previous outputs
        
        # Récupération des paramètres interactifs
        limit = str(limit_widget.value)
        max_spectres = str(max_spectres_widget.value)

        # Construction de la commande
        python_exe = sys.executable
        script_path = "../src/tools/dr5_downloader.py"
        cmd = [
            python_exe,
            script_path,
            "--limit", limit,
            "--max-spectres", max_spectres
        ]

        # Exécution avec meilleure gestion des erreurs
        print(f"Début du téléchargement avec {limit} plans et {max_spectres} spectres max...")
        try:
            result = subprocess.run(cmd, capture_output=True, text=True, check=True)
            print(result.stdout)
            print("Téléchargement terminé avec succès.")

            # Journalisation : enregistrement dans un log daté
            timestamp = datetime.datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
            log_filename = f"download_log_{timestamp}.txt"
            os.makedirs("../logs", exist_ok=True)
            with open(os.path.join("../logs", log_filename), "w", encoding="utf-8") as f:
                f.write(result.stdout)
            print(f"Log sauvegardé dans ../logs/{log_filename}")

        except subprocess.CalledProcessError as e:
            print("Une erreur s'est produite pendant le téléchargement :")
            print(e.stderr)
            # Sauvegarde aussi le log d'erreur
            timestamp = datetime.datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
            log_filename = f"error_log_{timestamp}.txt"
            os.makedirs("../logs", exist_ok=True)
            with open(os.path.join("../logs", log_filename), "w", encoding="utf-8") as f:
                f.write(e.stderr or "Aucune sortie d'erreur capturée.")
            print(f"Log d'erreur sauvegardé dans ../logs/{log_filename}")

run_button.on_click(on_run_clicked)


IntText(value=5, description='Plans :')

IntText(value=500, description='Spectres :')

Button(button_style='success', description='Lancer le téléchargement', style=ButtonStyle())

Output()

# 🧹 Option de Nettoyage
Il peut arriver que tu souhaites nettoyer entièrement le répertoire ``data/raw/`` avant de relancer un téléchargement, notamment pour repartir d’un dossier vierge ou libérer de l’espace.

Cette option de nettoyage intègre deux niveaux de protection pour éviter toute suppression accidentelle :

- Confirmation interactive :
Un bouton de confirmation via un widget (ou une autre méthode comme input()) demande explicitement ton accord avant d’exécuter la suppression.

- Sauvegarde automatique :
Avant de vider le répertoire, le script réalise une copie complète de data/raw/ dans un dossier temporaire nommé data/raw_backup/.

Cela te permet de restaurer tes spectres si jamais tu as validé la suppression par erreur.

**Important :**

- Le dossier de sauvegarde est écrasé à chaque nouvelle exécution de l’option.

- Tu peux restaurer des fichiers en récupérant les spectres depuis data/raw_backup/ tant que tu n’as pas relancé un nettoyage.

**Pourquoi utiliser cette option ?**

- Repartir sur un lot de spectres neuf sans mélange avec d’anciens fichiers.
- Tester un workflow complet à partir d’un état initial propre.
- Gagner en traçabilité et éviter les erreurs lors de sessions consécutives de téléchargement.

In [ ]:
import ipywidgets as widgets
import shutil, os
from IPython.display import display

confirm = widgets.ToggleButtons(
    options=["Non", "Oui"],
    description='Confirmer suppr. ?',
    disabled=False
)
display(confirm)

def on_change(change):
    if change['new'] == "Oui":
        backup_dir = "../data/raw_backup/"
        try:
            shutil.copytree("../data/raw/", backup_dir, dirs_exist_ok=True)
            shutil.rmtree("../data/raw/")
            os.makedirs("../data/raw/")
            print("data/raw vidé et sauvegardé dans raw_backup/")
        except Exception as e:
            print(f"Erreur : {e}")

confirm.observe(on_change, names='value')
